In [1]:
# 0: Imports / Setup

import unittest
from driver import (
    Expr, Var, Type, Pi, Lambda, App, Nat, Zero, Succ, ElimNat,
    Environment, type_check, eval_expr, alpha_equal, substitute
)

# Define a helper function to convert natural numbers to integers for readability
def nat_to_int(nat_expr):
    count = 0
    while isinstance(nat_expr, Succ):
        count += 1
        nat_expr = nat_expr.e
    return count

# Define a helper function to print test results
def print_test_result(test_name, success, message=""):
    status = "PASSED" if success else "FAILED"
    print(f"Test {test_name}: {status}")
    if message:
        print(f"  -> {message}")
    print("-" * 50)


In [12]:
# 1: Alpha Equivalence

def test_alpha_equivalence():
    test_name = "Alpha Equivalence"

    # Equivalent expressions
    expr1 = Lambda('x', Nat(), Var('x'))
    expr2 = Lambda('y', Nat(), Var('y'))

    # Non-equivalent expressions
    expr3 = Lambda('x', Nat(), Var('x'))
    expr4 = Lambda('y', Nat(), Var('x'))  # Incorrectly references 'x'

    # Equivalent expressions:
    try:
        result_eq = alpha_equal(expr1, expr2)
        if result_eq:
            alpha_eq_passed = True
            alpha_eq_message = ""
        else:
            alpha_eq_passed = False
            alpha_eq_message = f"Expressions {expr1} and {expr2} should be alpha-equivalent."
    except Exception as e:
        alpha_eq_passed = False
        alpha_eq_message = str(e)

    # Non-equivalent expressions:
    try:
        result_ne = alpha_equal(expr3, expr4)
        if not result_ne:
            alpha_ne_passed = True
            alpha_ne_message = ""
        else:
            alpha_ne_passed = False
            alpha_ne_message = f"Expressions {expr3} and {expr4} should not be alpha-equivalent."
    except Exception as e:
        alpha_ne_passed = False
        alpha_ne_message = str(e)

    # Aggregate Results
    success = alpha_eq_passed and alpha_ne_passed
    messages = []
    if not alpha_eq_passed:
        messages.append(alpha_eq_message)
    if not alpha_ne_passed:
        messages.append(alpha_ne_message)

    print_test_result(test_name, success, "\n".join(messages))

test_alpha_equivalence()


Test Alpha Equivalence: PASSED
--------------------------------------------------


In [2]:
# 2:  Identity Function

def test_identity_function():
    test_name = "Identity Function"

    # Define the polymorphic identity function
    id_expr = Lambda('A', Type(), Lambda('x', Var('A'), Var('x')))

    # Type check the identity function
    try:
        tau_id = type_check([], id_expr)
        expected_type = Pi('A', Type(), Pi('x', Var('A'), Var('A')))
        if alpha_equal(tau_id, expected_type):
            type_check_passed = True
            type_message = ""
        else:
            type_check_passed = False
            type_message = f"Expected type {expected_type}, got {tau_id}"
    except TypeError as e:
        type_check_passed = False
        type_message = str(e)

    # Define an application of the identity function to Nat
    try:
        id_nat = App(id_expr, Nat())
        tau_id_nat = type_check([], id_nat)
        expected_tau_id_nat = Pi('x', Nat(), Nat())
        if alpha_equal(tau_id_nat, expected_tau_id_nat):
            type_app_passed = True
            app_message = ""
        else:
            type_app_passed = False
            app_message = f"Expected type {expected_tau_id_nat}, got {tau_id_nat}"
    except TypeError as e:
        type_app_passed = False
        app_message = str(e)

    # Apply id_nat to Zero
    try:
        id_nat_zero = App(id_nat, Zero())
        tau_id_nat_zero = type_check([], id_nat_zero)
        expected_tau_id_nat_zero = Nat()
        if alpha_equal(tau_id_nat_zero, expected_tau_id_nat_zero):
            type_eval_passed = True
            eval_message = ""
        else:
            type_eval_passed = False
            eval_message = f"Expected type {expected_tau_id_nat_zero}, got {tau_id_nat_zero}"
    except TypeError as e:
        type_eval_passed = False
        eval_message = str(e)

    # Evaluate id_nat_zero
    try:
        result = eval_expr(id_nat_zero)
        expected_result = Zero()
        if alpha_equal(result, expected_result):
            eval_passed = True
            eval_result_message = ""
        else:
            eval_passed = False
            eval_result_message = f"Expected result {expected_result}, got {result}"
    except Exception as e:
        eval_passed = False
        eval_result_message = str(e)

    # Aggregate Results
    success = type_check_passed and type_app_passed and type_eval_passed and eval_passed
    messages = []
    if not type_check_passed:
        messages.append(type_message)
    if not type_app_passed:
        messages.append(app_message)
    if not type_eval_passed:
        messages.append(eval_message)
    if not eval_passed:
        messages.append(eval_result_message)

    print_test_result(test_name, success, "\n".join(messages))

test_identity_function()


Test Identity Function: PASSED
--------------------------------------------------


In [3]:
# 3: Constant Function

def test_constant_function():
    test_name = "Constant Function"

    # Define a constant function: const = lambda (A : *) . lambda (x : A) . lambda (y : A). x
    const_expr = Lambda('A', Type(), Lambda('x', Var('A'),
                       Lambda('y', Var('A'), Var('x'))))

    # Type check the constant function
    try:
        tau_const = type_check([], const_expr)
        expected_type = Pi('A', Type(),
                           Pi('x', Var('A'),
                              Pi('y', Var('A'), Var('A'))))
        if alpha_equal(tau_const, expected_type):
            type_check_passed = True
            type_message = ""
        else:
            type_check_passed = False
            type_message = f"Expected type {expected_type}, got {tau_const}"
    except TypeError as e:
        type_check_passed = False
        type_message = str(e)

    # Apply const to Nat and two Zero
    try:
        const_nat = App(App(const_expr, Nat()), Zero())
        const_nat_zero = App(const_nat, Zero())
        tau_const_nat_zero = type_check([], const_nat_zero)
        expected_tau = Nat()
        if alpha_equal(tau_const_nat_zero, expected_tau):
            type_eval_passed = True
            eval_message = ""
        else:
            type_eval_passed = False
            eval_message = f"Expected type {expected_tau}, got {tau_const_nat_zero}"
    except TypeError as e:
        type_eval_passed = False
        eval_message = str(e)

    # Evaluate const Nat 0
    try:
        result = eval_expr(const_nat_zero)
        expected_result = Zero()
        if alpha_equal(result, expected_result):
            eval_passed = True
            eval_result_message = ""
        else:
            eval_passed = False
            eval_result_message = f"Expected result {expected_result}, got {result}"
    except Exception as e:
        eval_passed = False
        eval_result_message = str(e)

    # Aggregate Results
    success = type_check_passed and type_eval_passed and eval_passed
    messages = []
    if not type_check_passed:
        messages.append(type_message)
    if not type_eval_passed:
        messages.append(eval_message)
    if not eval_passed:
        messages.append(eval_result_message)

    print_test_result(test_name, success, "\n".join(messages))

test_constant_function()


Test Constant Function: PASSED
--------------------------------------------------


In [4]:
# 4: Pi Type

def test_pi_type():
    test_name = "Pi Type"

    # Define a Pi-type: (A : *) -> (B : A) -> A
    pi_type = Pi('A', Type(), Pi('B', Var('A'), Var('A')))

    # Type check the Pi-type
    try:
        tau_pi = type_check([], pi_type)
        expected_type = Type()
        if alpha_equal(tau_pi, expected_type):
            type_check_passed = True
            type_message = ""
        else:
            type_check_passed = False
            type_message = f"Expected type {expected_type}, got {tau_pi}"
    except TypeError as e:
        type_check_passed = False
        type_message = str(e)

    print_test_result(test_name, type_check_passed, type_message)

test_pi_type()


Test Pi Type: PASSED
--------------------------------------------------


In [5]:
# 5: Lambda Type Mismatch

def test_lambda_type_mismatch():
    test_name = "Lambda Type Mismatch"

    # Define a lambda that returns a Nat: lambda (x : Nat). 0
    bad_lambda = Lambda('x', Nat(), Zero())

    # Type check should pass since Zero() is Nat and lambda expects to return Nat
    try:
        tau_lambda = type_check([], bad_lambda)
        expected_type = Pi('x', Nat(), Nat())
        if alpha_equal(tau_lambda, expected_type):
            type_check_passed = True
            type_message = ""
        else:
            type_check_passed = False
            type_message = f"Expected type {expected_type}, got {tau_lambda}"
    except TypeError as e:
        type_check_passed = False
        type_message = str(e)

    print_test_result(test_name, type_check_passed, type_message)

test_lambda_type_mismatch()

Test Lambda Type Mismatch: PASSED
--------------------------------------------------


In [6]:
# 6: Application Type Error

def test_application_type_error():
    test_name = "Application Type Error"

    # Attempt to apply Zero (which is Nat) as if it were a function
    application = App(Zero(), Zero())

    # Type checking should raise a TypeError
    try:
        type_check([], application)
        type_check_passed = False
        type_message = "Expected a TypeError but type checking succeeded."
    except TypeError as e:
        type_check_passed = True
        type_message = f"Caught expected TypeError: {e}"

    print_test_result(test_name, type_check_passed, type_message)

test_application_type_error()

Test Application Type Error: PASSED
  -> Caught expected TypeError: Attempting to apply non-function: Zero()
--------------------------------------------------


In [7]:
# 7: Succ Evaluation

# Cell 7: Test Case - Succ Evaluation

def test_succ_evaluation():
    test_name = "Succ Evaluation"

    # Define succ(0)
    succ_zero = Succ(Zero())

    # Type check succ_zero
    try:
        tau_succ_zero = type_check([], succ_zero)
        expected_type = Nat()
        if alpha_equal(tau_succ_zero, expected_type):
            type_check_passed = True
            type_message = ""
        else:
            type_check_passed = False
            type_message = f"Expected type {expected_type}, got {tau_succ_zero}"
    except TypeError as e:
        type_check_passed = False
        type_message = str(e)

    # Evaluate succ_zero
    try:
        result = eval_expr(succ_zero)
        expected_result = Succ(Zero())
        if alpha_equal(result, expected_result):
            eval_passed = True
            eval_message = ""
        else:
            eval_passed = False
            eval_message = f"Expected result {expected_result}, got {result}"
    except Exception as e:
        eval_passed = False
        eval_message = str(e)

    # Aggregate Results
    success = type_check_passed and eval_passed
    messages = []
    if not type_check_passed:
        messages.append(type_message)
    if not eval_passed:
        messages.append(eval_message)

    print_test_result(test_name, success, "\n".join(messages))

test_succ_evaluation()

Test Succ Evaluation: PASSED
--------------------------------------------------


In [8]:
# 8: ElimNat Zero

def test_elim_nat_zero():
    test_name = "ElimNat Zero"

    # Define elimNat e1 e2 e3 0
    e1 = Pi('x', Nat(), Type())
    e2 = Var('e2')  # Assume e2 has type e1 0, which should be Type()
    # Define e3: Inductive step
    e3 = Lambda(
        'x', 
        Nat(), 
        Lambda(
            '_', 
            App(Var('e1'), Var('x')),
            App(
                App(Var('e1'), Var('x')),
                App(
                    App(
                        App(Var('elimNat'), Var('e1')),
                        Var('e2')
                    ),
                    Var('e3')
                )
            )
        )
    )
    elim_nat_zero = ElimNat(e1, e2, e3, Zero())

    # Type checking should raise an error because e2 and e3 are unbound variables
    try:
        type_check([], elim_nat_zero)
        type_check_passed = False
        type_message = "Expected a TypeError due to unbound variables, but type checking succeeded."
    except TypeError as e:
        type_check_passed = True
        type_message = f"Caught expected TypeError: {e}"

    print_test_result(test_name, type_check_passed, type_message)

test_elim_nat_zero()

Test ElimNat Zero: PASSED
  -> Caught expected TypeError: Motive of elimNat is not of type ℕ → ⋆: Pi(x='x', tau1=Nat(), tau2=Type())
--------------------------------------------------


In [10]:
# 9: Type Star
def test_type_star():
    test_name = "Type Star"

    # Type-check Type
    try:
        tau_type = type_check([], Type())
        expected_type = Type()
        if alpha_equal(tau_type, expected_type):
            type_check_passed = True
            type_message = ""
        else:
            type_check_passed = False
            type_message = f"Expected type {expected_type}, got {tau_type}"
    except TypeError as e:
        type_check_passed = False
        type_message = str(e)

    print_test_result(test_name, type_check_passed, type_message)

test_type_star()

Test Type Star: PASSED
--------------------------------------------------


In [ ]:
# 10: Nat Type

def test_nat_type():
    test_name = "Nat Type"

    # Type-check Nat
    try:
        tau_nat = type_check([], Nat())
        expected_type = Type()
        if alpha_equal(tau_nat, expected_type):
            type_check_passed = True
            type_message = ""
        else:
            type_check_passed = False
            type_message = f"Expected type {expected_type}, got {tau_nat}"
    except TypeError as e:
        type_check_passed = False
        type_message = str(e)

    print_test_result(test_name, type_check_passed, type_message)

test_nat_type()

In [11]:
# 11: ElimNat Successor
def test_elim_nat_successor():
    test_name = "ElimNat Successor"

    # Define elimNat e1 e2 e3 (succ 0)
    e1 = Pi('x', Nat(), Type())
    e2 = Var('e2')  # Base case
    # Define e3: Inductive step
    e3 = Lambda(
        'x', 
        Nat(), 
        Lambda(
            '_', 
            App(Var('e1'), Var('x')),
            App(
                App(Var('e1'), Var('x')),
                App(
                    App(
                        App(Var('elimNat'), Var('e1')),
                        Var('e2')
                    ),
                    Var('e3')
                )
            )
        )
    )
    target = Succ(Zero())
    elim_nat_succ = ElimNat(e1, e2, e3, target)

    # Type checking should raise an error due to unbound variables
    try:
        type_check([], elim_nat_succ)
        type_check_passed = False
        type_message = "Expected a TypeError due to unbound variables, but type checking succeeded."
    except TypeError as e:
        type_check_passed = True
        type_message = f"Caught expected TypeError: {e}"

    print_test_result(test_name, type_check_passed, type_message)

test_elim_nat_successor()

Test ElimNat Successor: PASSED
  -> Caught expected TypeError: Motive of elimNat is not of type ℕ → ⋆: Pi(x='x', tau1=Nat(), tau2=Type())
--------------------------------------------------
